# GPA - Geometric Pose Affordance: 3D Human Pose with Scene Constraints

### Objective

__Explore different models__

### Potential Implementation

- Parameter tuning
- Cross validation
- Plotting

### Control Panel

In [1]:
run_linear_regression = False
run_random_forest_regressor = False
run_decision_tree_regressor = False
run_ada_boost_regressor = False
run_logistic_regression = False
run_ridge = False
run_sdg_regressor = False
run_lars = False
run_lasso_lars = False
run_mlp_regressor = False

### Import Libraries

In [2]:
import numpy as np

### Load Data

In [3]:
gt_3d_train_rel = np.load('train/gt_3d_train_rel.npy')
gt_2d_test = np.load('test/gt_2d_test.npy')
gt_2d_train = np.load('train/gt_2d_train.npy')
gt_2d_val = np.load('validation/gt_2d_val.npy')
gt_3d_val_rel = np.load('validation/gt_3d_val_rel.npy')

### Define X and Y

In [4]:
X = np.reshape(gt_2d_train, (gt_2d_train.shape[0], 16 * 2))
Y = np.reshape(gt_3d_train_rel, (gt_3d_train_rel.shape[0], 15 * 3))
X_val = np.reshape(gt_2d_val, (gt_2d_val.shape[0], 16 * 2))
Y_val = np.reshape(gt_3d_val_rel, (gt_3d_val_rel.shape[0], 15 * 3))

### Define Error Calculation Function

Error in 3D Euclidean Space between joint prediction and joint ground truth

In [5]:
def MPJPE_error(gt, pred): ## both prediction and ground truth in N*15*3, I will use this function to calculate final error and rank
    error = (gt - pred) ** 2
    pjpe = np.sum(error, 2) ** (1 / 2)
    mpjpe = np.mean(pjpe)
    return mpjpe

### Linear Regression Model

Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

Linear Regression fits a linear model with coefficients w = (w1, ..., wp) to minimize the residual sum of squares between the observed targets in the dataset, and the targets predicted by the linear approximation.

In [6]:
from sklearn.linear_model import LinearRegression

if run_linear_regression:
    regr_1 = linear_model.LinearRegression()
    regr_1.fit(X, Y)
    Y_val_pred_1 = regr_1.predict(X_val)
    Y_val_pred_reshape_1 = np.reshape(Y_val_pred_1, (Y_val_pred_1.shape[0], 15, 3))
    print(MPJPE_error(gt_3d_val_rel, Y_val_pred_reshape_1))

Best Error: __105.8613803487669__

### Random Forest Regressor Model

Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

A random forest is a meta estimator that fits a number of classifying decision trees on vairous sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. The sub-sample size is controlled with the max_samples parameter if bootstrap = True (default), otherwise the whole dataset is used to build each tree.

__Takes forever to run__

In [7]:
from sklearn.ensemble import RandomForestRegressor

if run_random_forest_regressor:
    regr_2 = RandomForestRegressor()
    regr_2.fit(X, Y)
    Y_val_pred_2 = regr_2.predict(X_val)
    Y_val_pred_reshape_2 = np.reshape(Y_val_pred_2, (Y_val_pred_2.shape[0], 15, 3))
    print(Y_val_pred_reshape_2.shape)
    print(MPJPE_error(gt_3d_val_rel, Y_val_pred_reshape_2))

### Decision Tree Regressor Model

Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor

In [8]:
from sklearn.tree import DecisionTreeRegressor

if run_decision_tree_regressor:
    regr_3 = DecisionTreeRegressor(random_state = 0)
    regr_3.fit(X, Y)
    Y_val_pred_3 = regr_3.predict(X_val)
    Y_val_pred_reshape_3 = np.reshape(Y_val_pred_3, (Y_val_pred_3.shape[0], 15, 3))
    print(MPJPE_error(gt_3d_val_rel, Y_val_pred_reshape_3))

Best Error: __139.53805192206954__

### Ada Boost Regressor Model

Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html

An AdaBoost regressor is a meta-estimator that begins by fitting a regressor on the original dataset and then fits additional copies of the regressor on the same dataset but where the weights of instances are adjusted according to the error of the current prediction. As such, subsequent regressors focus more on difficult cases.

__ValueError: bad input shape (222514, 45)__

In [9]:
from sklearn.ensemble import AdaBoostRegressor

if run_ada_boost_regressor:
    regr_4 = AdaBoostRegressor(random_state = 0, n_estimators = 100)
    regr_4.fit(X, Y)
    Y_val_pred_4 = regr_4.predict(X_val)
    Y_val_pred_reshape_4 = np.reshape(Y_val_pred_4, (Y_val_pred_4.shape[0], 15, 3))
    print(MPJPE_error(gt_3d_val_rel, Y_val_pred_reshape_4))

### [Logistic Regression Model](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

This is for documentation only; logistic regression is actually a classifier.

**bad input shape (222514, 45)**

In [10]:
from sklearn.linear_model import LogisticRegression

if run_logistic_regression:
    regr_5 = LogisticRegression().fit(X, Y)
    Y_val_pred_5 = regr_5.predict(X_val)
    Y_val_pred_reshape_5 = np.reshape(Y_val_pred_5, (Y_val_pred_5.shape[0], 15, 3))
    print(MPJPE_error(gt_3d_val_rel, Y_val_pred_reshape_5))

### [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)

Linear least squares with l2 regularization.

In [11]:
from sklearn.linear_model import Ridge

if run_ridge:
    regr_6 = Ridge().fit(X, Y)
    Y_val_pred_6 = regr_6.predict(X_val)
    Y_val_pred_reshape_6 = np.reshape(Y_val_pred_6, (Y_val_pred_6.shape[0], 15, 3))
    print(MPJPE_error(gt_3d_val_rel, Y_val_pred_reshape_6))

Best Error: **105.86082463709646**

### [SDGRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html)

Linear model fitted by minimizing a regularized empirical loss with SGD

**ValueError: bad input shape (222514, 45)**

In [12]:
from sklearn.linear_model import SGDRegressor

if run_sdg_regressor:
    regr_7 = SGDRegressor().fit(X, Y)
    Y_val_pred_7 = regr_7.predict(X_val)
    Y_val_pred_reshape_7 = np.reshape(Y_val_pred_7, (Y_val_pred_7.shape[0], 15, 3))
    print(MPJPE_error(gt_3d_val_rel, Y_val_pred_reshape_7))

### [Lars](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lars.html)

Least Angle Regression model a.k.a. LAR.

In [13]:
from sklearn.linear_model import Lars

if run_lars:
    regr_8 = Lars().fit(X, Y)
    Y_val_pred_8 = regr_8.predict(X_val)
    Y_val_pred_reshape_8 = np.reshape(Y_val_pred_8, (Y_val_pred_8.shape[0], 15, 3))
    print(MPJPE_error(gt_3d_val_rel, Y_val_pred_reshape_8))

Best error: **452862635248192.9**

### [LassoLars](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLars.html)

Lasso model fit with Least Angle Regression a.k.a. Lars.

It is a Linear Model trained with an L1 prior as regularizer.

In [14]:
from sklearn.linear_model import LassoLars

if run_lasso_lars:
    regr_9 = Lars().fit(X, Y)
    Y_val_pred_9 = regr_9.predict(X_val)
    Y_val_pred_reshape_9 = np.reshape(Y_val_pred_9, (Y_val_pred_9.shape[0], 15, 3))
    print(MPJPE_error(gt_3d_val_rel, Y_val_pred_reshape_9))

Best error: **452862635248192.9**

### [MLPRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html)

Multi-layer Perceptron regressor.

This model optimizes the squared-loss using LBFGS or stochastic gradient descent.

In [15]:
from sklearn.neural_network import MLPRegressor

if run_mlp_regressor:
    regr_10 = Lars().fit(X, Y)
    Y_val_pred_10 = regr_10.predict(X_val)
    Y_val_pred_reshape_10 = np.reshape(Y_val_pred_10, (Y_val_pred_10.shape[0], 15, 3))
    print(MPJPE_error(gt_3d_val_rel, Y_val_pred_reshape_10))

Best error: **452862635248192.9**